In [6]:
import torch
import torch.nn as nn
from initValues import config

hi


In [5]:
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


# Upsale the spatial size by a factor of 2
def upBlock(inPlanes, outPlanes):
    block = nn.Sequential(
        nn.Upsample(scale_factor=2, mode='nearest'),
        ConvBlock(inPlanes, outPlanes, kernel_size = 3, stride = 1)
        # conv3x3(in_planes, out_planes),
        # nn.BatchNorm2d(out_planes),
        # nn.ReLU(True))
    )
    return block


class ResBlock(nn.Module):
    def __init__(self, channelNum):
        super(ResBlock, self).__init__()
        self.block = nn.Sequential(
            ConvBlock(channelNum, channelNum, kernel_size=3, stride=1),
            # conv3x3(channel_num, channel_num),
            # nn.BatchNorm2d(channel_num),
            # nn.ReLU(True),
            conv3x3(channelNum, channelNum),
            nn.BatchNorm2d(channelNum))
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        out = self.block(x)
        out += residual
        out = self.relu(out)
        return out

class CA_NET(nn.Module):
  def __init__(self):
    super(CA_NET,self).__init__()
    self.t_dim = config.TEXT.DIMENSION
    self.c_dim = config.GAN.CONDITION_DIM
    self.fc = nn.Linear(self.t_dim, self.c_dim * 2, bias=True)
    self.relu = nn.ReLU()

  def encode(self, text_embedding):
    x = self.relu(self.fc(text_embedding))
    mu = x[:, :self.c_dim]
    logvar = x[:, self.c_dim:]
    return mu, logvar

  def reparametrize(self, mu, logvar):
    std = logvar.mul(0.5).exp_()
    if config.CUDA:
      eps = torch.cuda.FloatTensor(std.size(), requires_grad = True).normal_()
    else:
      eps = torch.FloatTensor(std.size(), requires_grad = True).normal_()
    return eps.mul(std).add_(mu)

  def forward(self, text_embedding):
     mu, logvar = self.encode(text_embedding)
     c_code = self.reparametrize(mu, logvar)
     return c_code, mu, logvar


class Stage1_Gen(nn.Module):
  def __init__(self):
    super(Stage1_GAN, self).__init__()
    self.gfDim = config.GAN.GF_DIM * 8
    self.efDim = config.GAN.CONDITION_DIM
    self.zDim = config.Z_DIM
    self.defineModel()

  def defineModel(self):
    inp = self.zDim + self.efDim
    ngf - self.gfDim
    self.caNet = CA_NET()

    self.net = nn.Sequential(
        nn.Linear(inp, ngf * 4 * 4, bias = False),
        nn.BatchNorm1d(ngf * 4 * 4),
        nn.ReLU(True)
    )
    self.upSam = nn.Sequential(
        upBlock(ngf, ngf//2),
        upBlock(ngf // 2, ngf // 4),
        upBlock(ngf // 4, ngf // 8),
        upBlock(ngf // 8, ngf // 16)
    )
    # self.upSam1 = upBlock(ngf, ngf//2)
    # self.upSam2 = upBlock(ngf // 2, ngf // 4)
    # self.upSam3 = upBlock(ngf // 4, ngf // 8)
    # self.upSam4 = upBlock(ngf // 8, ngf // 16)
    
    self.img = nn.Sequential(
        conv3x3(ngf // 16, 3),
        nn.Tanh()
    )
  
  def forward(self, textEmbedding, noise):
    cCode , mu, logvar = self.caNet(textEmbedding)
    zCCode = torch.cat((noise, cCode), 1)
    hCode = self.net(zCCode)

    hCode = hCode.view(-1, self.gfDim, 4, 4)
    hCode = self.upSam(hCode)

    fakeImg = self.img(hCode)
    return None, fakeImg, mu, logvar

class ConvBlock(nn.Module):
  def __init__(self, inpC, outC, kernel_size = 4, stride = 2, padding = 1, bias = False, BN = True, leaky = False):
    super(ConvBlock, self).__init__()
    self.BN = BN
    self.leaky = leaky
    self.conv = nn.Conv2d(inpC, outC, kernel_size, stride, padding, bias)
    self.batchNorm = nn.BatchNorm2d(outC)
    self.relu = nn.ReLU(inplace=True)
    self.leakyRelu = nn.LeakyReLU(0.2, inplace = True)

  def forward(self, x):
    out = self.conv(x)
    if BN:
      out = self.batchNorm(out)
    out = self.leakyRelu(out) if self.leaky else self.relu(out)
    return out

class Stage1_Dis(nn.Module):
  def __init__(self):
    super(Stage1_Dis, self).__init__()
    self.dfDim = config.GAN.DF_DIM
    self.efDim = config.GAN.CONDITION_DIM
    self.defineModel()

  def defineModel(self):
    ndf, nef = self.dfDim, self.efDim

    self.encodeImg = nn.Sequential(
        ConvBlock(3, ndf, BN=False, leaky = True),
        #(ndf) x 32 x 32
        ConvBlock(ndf, ndf*2, leaky = True),
        #(ndf*2) x 16 x 16
        ConvBlock(ndf*2, ndf * 4, leaky = True),
        #(ndf*4) x 8 x 8
        ConvBlock(ndf * 4, ndf * 8, leaky = True)
        #(ndf*8) x 4 x 4
    )

    self.getCondLogits = D_GetLogits(ndf, nef)

  def forward(self, image):
    imgEmbedding = self.encodeImg(image)
    return imgEmbedding

class Stage2_Gen(nn.Module):
  def __init__(self, Stage1_Gen):
    super(Stage2_Gen, self).__init__()
    self.gfDim = config.GAN.GF_DIM
    self.efDim = config.GAN.CONDITION_DIM
    self.zDim = config.Z_DIM
    self.stage1Gen = Stage1_Gen

    for param in self.stage1Gen.parameters():
      param.requires_grad = False
    self.defineModel()

  def _make_layer(self, block, channelNum):
    layers = []
    for i in range(config.GAN.R_NUM):
      layers.append(block(channelNum))
    return nn.Sequential(*layers)

  def defineModel(self):
    ngf = self.gfDim

    self.caNet = CA+NET()

    self.encoder = nn.Sequential(
        ConvBlock(3, ngf, kernel_size=3, stride = 1, BN = False),
        ConvBlock(ngf, ngf * 2),
        ConvBlock(ngf * 2, ngf * 4)
    )

    self.hrJoint = nn.Sequential(
        ConvBlock(self.efDim + ngf * 4, ngf * 4, kernel_size = 3, stride = 1)
    )

    self.residual = self._make_layer(ResBlock, ngf * 4)

    self.upSam = nn.Sequential(
        upBlock(ngf * 4, ngf * 2),
        upBlock(ngf * 2, ngf),
        upBlock(ngf, ngf // 2),
        upBlock(ngf // 2, ngf // 4)
    )

    self.img = nn.Sequential(
        nn.Conv2d(ngf // 4, 3, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.Tanh()
    )

  def forward(self, textEmbedding, noise):
    _, stage1_Img, _, _ = self.Stage1_Gen(textEmbedding, noise)
    stage1_Img = stage1_Img.detach()
    encodedImg = self.encoder(stage1_Img)

    cCode , mu, logvar = self.caNet(textEmbedding)
    cCode = cCode.view(-1, self.efDim, 1, 1)
    cCode = cCode.repeat(1, 1, 16, 16)
    icCode = torch.cat([encodedImg, cCode], 1)
    hCode = self.hrJoint(icCode)
    hCode = self.residual(hCode)

    hCode = self.upSam(hCode)

    fakeImg = self.img(hCode)
    return stage1_Img, fakeImg, mu, logvar

class Stage2_Dis(nn.Module):
  def __init__(self):
    super(Stage2_Dis, self).__init__()
    self.dfDim = config.GAN.DF_DIM
    self.efDim = config.GAN.CONDITION_DIM
    self.defineModel()

  def defineModel(self):
    ndf, nef = self.dfDim, self.efDim
    self.encodeImg = nn.Sequential(
        ConvBlock(3, ndf, BN = False, leaky = True),
        #128 * 128 * (ndf)
        ConvBlock(ndf, ndf * 2, leaky = True),
        #64 x 64 x (ndf*2)
        ConvBlock(ndf * 2, ndf * 4, leaky = True),
        #32 x 32 x (ndf*4)
        ConvBlock(ndf * 4, ndf * 8, leaky = True),
        #16 x 16 x (ndf*8)
        ConvBlock(ndf * 8, ndf * 16, leaky = True),
        #8 x 8 x (ndf*16)
        ConvBlock(ndf * 16, ndf * 32, leaky = True),
        #4 x 4 x (ndf*32)

        ConvBlock(ndf * 32, ndf * 16, kernel_size = 3, stride=1, leaky = True),
        #4 x 4 x ndf * 16
        ConvBlock(ndf * 16, ndf * 8, kernel_size = 3, stride=1, leaky = True)
        #4 x 4 x ndf x 8
    )

    self.getCondLogits = D_GetLogits(ndf, nef, bcondition = True)
    self.getUnCondLogits = D_GetLogits(ndf, nef, bcondition = False)

  def forward(self, image):
    imgEmbedding = seld.encodeImg(image)
    return imgEmbedding